In [1]:
import wandb
import pandas as pd
from tqdm import tqdm_notebook
pd.set_option("display.max_columns",100)
import numpy as np
import pickle
import pytabchen as pytb

In [2]:
import yaml

In [3]:
# 显示所有列(参数设置为None代表显示所有行，也可以自行设置数字)
pd.set_option('display.max_columns',None)
# 显示所有行
pd.set_option('display.max_rows',None)
# 设置数据的显示长度，默认为50
pd.set_option('max_colwidth',200)
# 禁止自动换行(设置为Flase不自动换行，True反之)
pd.set_option('expand_frame_repr', False)

In [4]:
wandb_config = {
    "uid": "Shyann",
    "api_key": "07ec58f8e8315aa083a202292092d7e2ee9d43b4"
}

In [5]:
import wandb 
import os
os.environ['WANDB_API_KEY'] = wandb_config['api_key']

In [6]:
def get_runs_result(runs):
    result_list = []
    for run in tqdm_notebook(runs):
        result = {}
        result.update(run.summary._json_dict)
        model_config = {k: v for k, v in run.config.items()
                    if not k.startswith('_') and type(v) not in [list,dict]}
        result.update(model_config)
        result['Name'] = run.name
        result['path_id'] = run.path[-1]
        result_list.append(result)
    runs_df = pd.DataFrame(result_list)#.drop_duplicates(list(model_config.keys()))
    return runs_df

def get_df(sweep_name,sweep_dict):
    df_list = []
    for sweep_id in sweep_dict[sweep_name]:
#         sweep = api.sweep("Shyann/掌握程度/{}".format(sweep_id))
        sweep = api.sweep("Shyann/{}/{}".format(project_name,sweep_id))
        df = get_runs_result(sweep.runs)
        df_list.append(df)
    df = pd.concat(df_list)
#     print(df.shape)
    return df

def get_sweep_dict(project):
    '''获取sweep的id'''
    sweep_dict = {}
    for sweep in project.sweeps():
        if sweep.name not in sweep_dict:
            sweep_dict[sweep.name] = []
        sweep_dict[sweep.name].append(sweep.id)
    return sweep_dict

In [7]:
all_res = dict()
end = 0
end_ids = []
all_end_ids = []
for key in sweep_dict:
    if not key.startswith("assist2009_atktfix_forget_atc_0"):
        continue
    try:
        df = get_df(key, sweep_dict)
        print(f"ori df: {df.shape}")
        df = df.dropna(subset=["testauc"])
        print(f"df dropna: {df.shape}")
        all_res[key] = df
        df["Name"] = df["Name"].apply(
            lambda a: int(a.split("-")[-1]))  # 创建的任务名字有顺序
#         df["key"] = df["testauc"] - df["Name"]
#         sorted_df = df.sort_values("key", ascending=False)
        sorted_df = df.sort_values("testauc", ascending=False)

        testauco = sorted_df["testauc"].tolist()[0]
        testacco = sorted_df["testacc"].tolist()[0]
        best_epoch = sorted_df["best_epoch"].tolist()[0]

        testauc = round(testauco, 4)
        testacc = round(testacco, 4)
        best_epoch = int(best_epoch)

        topdf = df[round(df["testauc"], 4) == testauc].sort_values("Name")
        best_name = topdf["Name"].tolist()[0]
        best_path_id = topdf["path_id"].tolist()[0]
#         print(df.shape)
        print(key, sweep_dict[key], testauc, testacc,
              best_epoch, best_name, best_path_id, topdf.shape[0])

        end_cmd = f"/root/anaconda3/envs/lqq_ktmore/bin/wandb sweep Shyann/{project_name}/{sweep_dict[key][0]} --cancel"
        not_improve_num = len(df[df['Name'] >= best_name])
        if df.shape[0] > 300 and not_improve_num >= 100:
            end += 1
            print("end!!!")
            end_ids.append(end_cmd)
#         else:
        print(f"已经有 {not_improve_num} 轮没提升了")
        all_end_ids.append(end_cmd)
    except Exception as e:
        print(e)
        continue

NameError: name 'sweep_dict' is not defined

## 汇总emb结果

In [169]:
# api = wandb.Api()
# project_name = "kt_toolkits_emb"
# project = api.project(name=project_name)
# sweep_dict = get_sweep_dict(project)
# sweep_dict

In [19]:
api = wandb.Api(timeout=19)
project_name = "pykt_dbkt"
project = api.project(name=project_name)
sweep_dict = get_sweep_dict(project)
sweep_dict

{'assist2009_deepbkt_bayesian_forgetting_4': ['us3ebgtn'],
 'assist2009_deepbkt_bayesian_forgetting_3': ['i0ukp0ev'],
 'assist2009_deepbkt_bayesian_forgetting_2': ['y69jkfin'],
 'assist2009_deepbkt_bayesian_forgetting_1': ['yp4pmvsa'],
 'assist2009_deepbkt_bayesian_forgetting_0': ['zm3vr3dv'],
 'assist2009_deepbkt_augmentation_forgetting_4': ['xp0rpo6p'],
 'assist2009_deepbkt_augmentation_forgetting_3': ['nuxtv9k7'],
 'assist2009_deepbkt_augmentation_forgetting_2': ['karr35xn'],
 'assist2009_deepbkt_augmentation_forgetting_1': ['jyhkwfgi'],
 'assist2009_deepbkt_augmentation_forgetting_0': ['eq2utqft', '2tkq5r7a'],
 'atskhfdg': ['atskhfdg'],
 'assist2009_deepbkt_all_4': ['q99t9ek2'],
 'assist2009_deepbkt_all_3': ['e0l6ywvd'],
 'assist2009_deepbkt_all_2': ['ssvwtsyd'],
 'assist2009_deepbkt_all_1': ['p3ljm26m'],
 'assist2009_deepbkt_all_0': ['8p19y63j'],
 'assist2009_deepbkt_forgetting_4': ['gvwunjhm'],
 'assist2009_deepbkt_forgetting_3': ['rqh6h58l'],
 'assist2009_deepbkt_forgetting_2': 

In [20]:
result_save_dir = 'results'
os.makedirs(result_save_dir, exist_ok=True)

In [21]:
groups = set(["_".join(x.split("_")[:-1]) for x in list(sweep_dict.keys())])
groups

{'',
 'assist2009_deepbkt_all',
 'assist2009_deepbkt_augmentation',
 'assist2009_deepbkt_augmentation_bayesian',
 'assist2009_deepbkt_augmentation_forgetting',
 'assist2009_deepbkt_bayesian',
 'assist2009_deepbkt_bayesian_forgetting',
 'assist2009_deepbkt_forgetting',
 'assist2009_deepbkt_qid'}

In [22]:
def save_pickle(data, path):
    with open(path, "wb") as f:
        pickle.dump(data, f)

### 保存

In [ ]:
for group in tqdm_notebook(groups):
    if group not in ['assist2009_deepbkt_forgetting']:
        continue
    one_group_list = []
    for cv in range(5):
        key = f"{group}_{cv}"
        df = get_df(key, sweep_dict)
        one_group_list.append(df)
    save_pickle(one_group_list,os.path.join(result_save_dir,f"{group}.pkl"))

/root/anaconda3/envs/4s_dev/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/9 [00:00<?, ?it/s]

/root/anaconda3/envs/4s_dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/188 [00:00<?, ?it/s]

### 分析

In [13]:
import shutil

In [14]:
best_model_dir = 'data/best_models/assist2009_deepbkt_forgetting'
raw_model_dir = "/hw"
os.makedirs(best_model_dir,exist_ok=True)

In [15]:
# groups = ['assist2009_dkt_akt_vectoremb',
#           'assist2009_dkt_atktemb',
#           'assist2009_dkt_ektemb',
#           'assist2009_dkt_lpktemb',
#           'assist2009_dkt_akt_vectoremb',
#           'assist2009_dkt_atktemb',
#           'assist2009_dkt_ektemb',
#           'assist2009_dkt_lpktemb']
datasets = ["assist2009"]
emb_types = ['forgetting']
groups = []
models = ['deepbkt']
for dataset in datasets:
    for model in models:
        for emb in emb_types:
            groups.append(f"{dataset}_{model}_{emb}")
groups

['assist2009_deepbkt_forgetting']

In [16]:
cols = ['testauc', 'testacc', 'window_testauc', 'window_testacc']

In [41]:
#5-folds

In [33]:
one_group = pytb.load_pickle(f"results/algebra2005_lpkt_qid.pkl")
cv_results = [x.sort_values("testauc",ascending=False).iloc[0] for x in one_group]
df_one = pd.DataFrame(cv_results)

In [ ]:
# one-fold

In [17]:
one_group = pytb.load_pickle(f"results/assist2009_deepbkt_forgetting.pkl")
cv_results = one_group[0].sort_values("testauc",ascending=False).iloc[0]
df_one = pd.DataFrame(cv_results)

In [18]:
df_one

,68
d_ff,512
fold,0
seed,42
d_model,256
dropout,0.2
emb_type,forgetting
n_blocks,2
save_dir,deepbkt_tiaocan_assist2009
model_name,deepbkt
dataset_name,assist2009


In [42]:
df_one["window_testacc"].mean()

-1.0

In [41]:
df_one["testacc"].std()

0.0004098780306383821

In [45]:
model_paths = df_one["model_save_path"].tolist()
model_paths

['akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_3407_0_0.15_256_512_8_4_0.0001_1_1_dc66aa10-250d-4a1a-879f-80f80b13f6bd/qid_model.ckpt',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_224_1_0.05_256_256_4_4_1e-05_1_1_6681870f-d2e8-4ac6-bc08-adeb80ee8739/qid_model.ckpt',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_224_2_0.1_256_256_4_4_1e-05_1_1_01ece1cc-646f-4a39-aaaf-f6c443c522cd/qid_model.ckpt',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_3407_3_0.05_256_512_4_4_1e-05_1_1_d8d7d359-c536-41a4-94e0-3e6fa14dd211/qid_model.ckpt',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_3407_4_0.15_256_512_4_4_0.0001_1_1_bb456171-9504-477a-a77f-9a8998b3696b/qid_model.ckpt']

In [41]:
file_name = "aktvec_qid"

In [46]:
model_paths = [x.rstrip("qid_model.ckpt") for x in model_paths]
model_paths

['akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_3407_0_0.15_256_512_8_4_0.0001_1_1_dc66aa10-250d-4a1a-879f-80f80b13f6bd/',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_224_1_0.05_256_256_4_4_1e-05_1_1_6681870f-d2e8-4ac6-bc08-adeb80ee8739/',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_224_2_0.1_256_256_4_4_1e-05_1_1_01ece1cc-646f-4a39-aaaf-f6c443c522cd/',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_3407_3_0.05_256_512_4_4_1e-05_1_1_d8d7d359-c536-41a4-94e0-3e6fa14dd211/',
 'akt_vector_tiaocan_assist2009/assist2009_akt_vector_qid_akt_vector_tiaocan_assist2009_3407_4_0.15_256_512_4_4_0.0001_1_1_bb456171-9504-477a-a77f-9a8998b3696b/']

In [47]:
fpath = "/data/huangshuyan/shyann_dev/pykt-toolkit/examples/seedwandb/predict.yaml"
ftarget = "/data/huangshuyan/shyann_dev/pykt-toolkit/examples/seedwandb/{}.yaml".format(file_name)

In [48]:
with open(fpath,"r") as fin,\
    open(ftarget,"w") as fout:
    data = yaml.load(fin, Loader=yaml.FullLoader)
    name = ftarget.split('_')
    data['name'] = '_'.join(name[:4])
    data['parameters']['save_dir']['values'] = model_paths
    yaml.dump(data, fout)

In [96]:
df_one.to_csv('./results/assist2009_akt_vectorvec.forget_atc_lstmy_forget_atc')

In [183]:
cv_report_list = []
for group in tqdm_notebook(groups):
    one_group = pytb.load_pickle(f"data/wandb_result/{group}.pkl")
    cv_results = [x.sort_values("testauc",ascending=False).iloc[0] for x in one_group]
    df_one = pd.DataFrame(cv_results)
    cv_report = {'group':group}
    mean_report = df_one[cols].mean(axis=0).to_dict()
    std_report = df_one[cols].std(axis=0,ddof=0).to_dict()
    for k in mean_report:
        cv_report[k] = "{:.4f}±{:.4f}".format(mean_report[k], std_report[k])
    cv_report_list.append(cv_report)
    
    
    
#     for _,row in df_one.iterrows():
#         model_save_dir = os.path.join(raw_model_dir,os.path.dirname(row['model_save_path']))
#         dir_name = os.path.basename(model_save_dir)
#         new_save_dir = os.path.join(best_model_dir,group,f"{dir_name}_{row['fold']}")
#         print(f"model_save_dir is {model_save_dir}")
#         print(f"new_save_dir is {new_save_dir}")
#         print()
# #         model_save_dir = os.path.dirname(row['model_save_path'])
# #         shutil.copytree(model_save_dir,new_save_dir)

/root/anaconda3/envs/4s_dev/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/1 [00:00<?, ?it/s]

In [151]:
df_report = pd.DataFrame(cv_report_list)
df_report

NameError: name 'cv_report_list' is not defined

In [119]:
df_report = pd.DataFrame(cv_report_list)
df_report

,group,testauc,testacc,window_testauc,window_testacc
0,bridge2algebra2006_gkt_qid,0.8018±0.0007,0.8549±0.0003,-1.0000±0.0000,-1.0000±0.0000


In [59]:
df_report[cols]

,testauc,testacc,window_testauc,window_testacc
0,0.8083±0.0040,0.7926±0.0013,-1.0000±0.0000,-1.0000±0.0000


In [60]:
df_report

,group,testauc,testacc,window_testauc,window_testacc
0,statics2011_gkt_qid,0.8083±0.0040,0.7926±0.0013,-1.0000±0.0000,-1.0000±0.0000


## 其它

In [ ]:
def get_params(paramsf):
    lines = open(paramsf).read().split("\n")
    params, params_hy = [], []
    for i in range(0, len(lines)):
        cur = lines[i]
        pre = ""
        if i > 0:
            pre = lines[i-1]
        if pre == "":
            continue
        idx = cur.find("values")
        if idx != -1:
            key = pre.strip().replace(":", "")
            params.append(key)
            
            if key not in ["fold", "seed"]:
                params_hy.append(key)
    return params, params_hy


In [ ]:
import numpy as np

diffs = []
dataset_name = "all_wandbs"
for key in all_res:
    d = dict()    
    paramsf = os.path.join(dataset_name, key+".yaml")
    params, params_hy = get_params(paramsf)
#     params = sorted(params)
    df = all_res[key].dropna()
    if "testauc" not in df.keys():
        continue
#     print(df)
#     sorted_df = df.sort_values("testauc",ascending=False)
    
    pams = dict()
    for i, row in df.iterrows():
        fold, seed = row["fold"], row["seed"]
        params_hystr = "_".join([str(s) for s in row[params_hy].values])
        d.setdefault(params_hystr, dict())
        d[params_hystr].setdefault(fold, dict())
        d[params_hystr][fold].setdefault(seed, [])
        
        params_str = "_".join([str(s) for s in row[params].values])
        if params_str in pams: # 去重
            curauc = row["testauc"]
            if curauc != pams[params_str]:
                print(f"bad!! params: {params_str}, curauc: {curauc}, saveauc: {pams[params_str]}")
            continue
        pams[params_str] = row["testauc"]
        values = [row["testauc"], row["testacc"], row["window_testauc"], row["window_testacc"]]
        d[params_hystr][fold][seed] = values
    print(f"df: {df.shape}, len(pams): {len(pams)}")
        
    maxres = [0.0] * 8
    for params in d:
        aucs, accs, winaucs, winaccs = [], [], [], []
        for fold in d[params]:
            auc, acc, winauc, winacc = 0.0, 0.0, 0.0, 0.0
            for seed in d[params][fold]:
                curauc, curacc, curwinauc, curwinacc = d[params][fold][seed]
#                 print(f"params: {params}, fold: {fold}, seed: {seed}, len: {len(d[params][fold][seed])}")
                if curauc > auc:
                    auc, acc, winauc, winacc = curauc, curacc, curwinauc, curwinacc
#             print(f"params: {params}, fold: {fold}, len: {len(d[params][fold])}, auc: {auc}, acc: {acc}, winauc: {winauc}, winacc: {winacc}")
            aucs.append(auc)
            accs.append(acc)
            winaucs.append(winauc)
            winaccs.append(winacc)
            
        auc_mean, auc_std = round(np.mean(aucs), 4), round(np.std(aucs, ddof=0), 4)
        acc_mean, acc_std = round(np.mean(accs), 4), round(np.std(accs, ddof=0), 4)
        winauc_mean, winauc_std = round(np.mean(winaucs), 4), round(np.std(winaucs, ddof=0), 4)
        winacc_mean, winacc_std = round(np.mean(winaccs), 4), round(np.std(winaccs, ddof=0), 4)
                
#         print(f"\nparams: {params}, auc_mean: {auc_mean}, acc_mean: {acc_mean}, winauc_mean: {winauc_mean}, winacc_mean: {winacc_mean}")
#         print(f"    auc_std: {auc_std}, acc_std: {acc_std}, winauc_std: {winauc_std}, winacc_std: {winacc_std}")
        
#         print("="*20)
        
        if auc_mean > maxres[0]:
            maxres = [auc_mean, auc_std, acc_mean, acc_std, winauc_mean, winauc_std, winacc_mean, winacc_std]
            
    print(key, maxres)

        

In [ ]:
import numpy as np

diffs = []
dataset_name = "all_wandbs"
for key in all_res:
    d = dict()    
    paramsf = os.path.join(dataset_name, key+".yaml")
    params, params_hy = get_params(paramsf)
#     params = sorted(params)
    df = all_res[key].dropna()
    if "testauc" not in df.keys():
        continue
#     print(df)
#     sorted_df = df.sort_values("testauc",ascending=False)
    
    pams = dict()
    for i, row in df.iterrows():
        fold, seed = row["fold"], row["seed"]
        params_hystr = "_".join([str(s) for s in row[params_hy].values])
        d.setdefault(fold, dict())
        d[fold].setdefault(params_hystr, dict())
        d[fold][params_hystr].setdefault(seed, [])
        
        params_str = "_".join([str(s) for s in row[params].values])
        if params_str in pams: # 去重
            curauc = row["testauc"]
            if curauc != pams[params_str][0]:
                print(f"bad!! params: {params_str}, curauc: {curauc}, saveauc: {pams[params_str]}")
                print(row["Name"])
            continue
        pams[params_str] = [row["testauc"], row["Name"]]
        values = [row["testauc"], row["testacc"], row["window_testauc"], row["window_testacc"]]
        d[fold][params_hystr][seed] = values
    print(f"df: {df.shape}, len(pams): {len(pams)}")
        
    aucs, accs, winaucs, winaccs = [], [], [], []
    for fold in d:
        auc, acc, winauc, winacc = 0.0, 0.0, 0.0, 0.0
        for params in d[fold]:
            for seed in d[fold][params]:
                curauc, curacc, curwinauc, curwinacc = d[fold][params][seed]
#                 print(f"params: {params}, fold: {fold}, seed: {seed}, len: {len(d[params][fold][seed])}")
                if curauc > auc:
                    auc, acc, winauc, winacc = curauc, curacc, curwinauc, curwinacc
#             print(f"params: {params}, fold: {fold}, len: {len(d[params][fold])}, auc: {auc}, acc: {acc}, winauc: {winauc}, winacc: {winacc}")
        aucs.append(auc)
        accs.append(acc)
        winaucs.append(winauc)
        winaccs.append(winacc)
            
    auc_mean, auc_std = round(np.mean(aucs), 4), round(np.std(aucs, ddof=0), 4)
    acc_mean, acc_std = round(np.mean(accs), 4), round(np.std(accs, ddof=0), 4)
    winauc_mean, winauc_std = round(np.mean(winaucs), 4), round(np.std(winaucs, ddof=0), 4)
    winacc_mean, winacc_std = round(np.mean(winaccs), 4), round(np.std(winaccs, ddof=0), 4)
                
#         print(f"\nparams: {params}, auc_mean: {auc_mean}, acc_mean: {acc_mean}, winauc_mean: {winauc_mean}, winacc_mean: {winacc_mean}")
#         print(f"    auc_std: {auc_std}, acc_std: {acc_std}, winauc_std: {winauc_std}, winacc_std: {winacc_std}")
        
#         print("="*20)
            
    print(key, auc_mean, auc_std, acc_mean, acc_std, winauc_mean, winauc_std, winacc_mean, winacc_std)

        

In [ ]:
diffs = []
dataset_name = "all_wandbs"
for key in all_res:
#     if not key.startswith("xes_dkt_forget_atccatr"):
#         continue
    d = dict()
    d.setdefault(key, dict())
    keys = []
    paramsf = os.path.join(dataset_name, key+".yaml")
    params = get_params(paramsf)
    params = sorted(params)
    df = all_res[key].dropna()
    if "testauc" not in df.keys():
        continue
#     print(df)
    sorted_df = df.sort_values("testauc",ascending=False)
    
    top1 = sorted_df[0:1]
    for i, row in top1[params].iterrows():
        params_str = "_".join([str(s) for s in row.values])
    d[key].setdefault(params_str, [])
    print(params_str)

    for i, row in sorted_df.iterrows():
        try:
            testauc = row["testauc"]
            testacc = row["testacc"]
            best_epoch = row["best_epoch"]
            testauc = round(testauc, 4)
            testacc = round(testacc, 4)
            best_epoch = int(best_epoch)
            curparams = "_".join([str(s) for s in row[params].values])
    #         print(curparams)
            if params_str == curparams:
                d[key][params_str].append([testauc, row["Name"]])
        except Exception as e:
            print(key, row)
            print(e)
            import sys
            sys.exit()
    min_auc = d[key][params_str][-1][0]
    max_auc = d[key][params_str][0][0]
    print(len(d[key][params_str]))
#     min_auc = sorted_df["testauc"].tolist()[-1]
#     max_auc = sorted_df["testauc"].tolist()[0]
    maxdiff = round(max_auc-min_auc,4)
    print(key, min_auc, max_auc, maxdiff, df.shape[0])#, len(d[key][params_str]), df.shape[0])
    diffs.append(maxdiff)
sorted(diffs)

In [ ]:
# preend = "WANDB_API_KEY=dc95dfebd33a883d3bdd540b4396189aad828f59 wandb sweep kitty3/knowledge_tracing_models/"

# for key in sweep_dict:
#     val = sweep_dict[key]
#     cmd = preend + val[0] + " --cancel"
#     print(cmd)



In [ ]:
# paramsf = "./Statics2011_wandb/Statics2011_dkvmn_forget_atc.yaml"
# def get_top1_params(paramsf):
#     lines = open(paramsf).read().split("\n")
#     res = []
#     for i in range(0, len(lines)):
#         cur = lines[i]
#         pre = ""
#         if i > 0:
#             pre = lines[i-1]
#         if pre == "":
#             res.append(cur)
#             continue
#         idx = cur.find("values")
#         if idx != -1:
#             key = pre.strip().replace(":", "")
#             res.append(cur[:idx] + "values: " + "[" + str(top1[key].tolist()[0]) + "]")
#         else:
#             res.append(cur)
#     return res


In [ ]:
# for key in sweep_dict:
#     yaml = key + ".yaml"
    
#     df = get_df(key,sweep_dict)
#     sorted_df = df.sort_values("testauc",ascending=False)
#     top1 = sorted_df[0:1]
# top1

In [ ]:
# def download(sweep_dict):
#     d_data = dict()
#     for key in sweep_dict:
#         try:
#             df = get_df(key, sweep_dict)
#             d_data[key] = df
#         except:
#             continue
#     return d_data
# all_res = download(sweep_dict)

In [ ]:
# def get_params(paramsf):
#     lines = open(paramsf).read().split("\n")
#     res = []
#     for i in range(0, len(lines)):
#         cur = lines[i]
#         pre = ""
#         if i > 0:
#             pre = lines[i-1]
#         if pre == "":
#             continue
#         idx = cur.find("values")
#         if idx != -1:
#             key = pre.strip().replace(":", "")
#             res.append(key)
#     return res


In [ ]:
# diffs = []
# dataset_name = "all_wandbs"
# for key in all_res:
#     if not key.startswith("Statics2011"):
#         continue
#     d = dict()
#     d.setdefault(key, dict())
#     keys = []
#     paramsf = os.path.join(dataset_name, key+".yaml")
#     params = get_params(paramsf)
#     params = sorted(params)
#     df = all_res[key].dropna()
#     if "testauc" not in df.keys():
#         continue
# #     print(df)
#     sorted_df = df.sort_values("testauc",ascending=False)
# #     top1 = sorted_df[0:1]
# #     for i, row in top1[params].iterrows():
# #         params_str = "_".join([str(s) for s in row.values])
# #     d[key].setdefault(params_str, [])
# #     print(params_str)

# #     for i, row in sorted_df.iterrows():
# #         try:
# #             testauc = row["testauc"]
# #             testacc = row["testacc"]
# #             best_epoch = row["best_epoch"]
# #             testauc = round(testauc, 4)
# #             testacc = round(testacc, 4)
# #             best_epoch = int(best_epoch)
# #             curparams = "_".join([str(s) for s in row[params].values])
# #     #         print(curparams)
# #             if params_str == curparams:
# #                 d[key][params_str].append([testauc, row["Name"]])
# #         except Exception as e:
# #             print(key, row)
# #             print(e)
# #             import sys
# #             sys.exit()
# #     min_auc = d[key][params_str][-1][0]
# #     max_auc = d[key][params_str][0][0]
#     min_auc = sorted_df["testauc"].tolist()[-1]
#     max_auc = sorted_df["testauc"].tolist()[0]
#     maxdiff = round(max_auc-min_auc,4)
#     print(key, min_auc, max_auc, maxdiff, df.shape[0])#, len(d[key][params_str]), df.shape[0])
#     diffs.append(maxdiff)
# sorted(diffs)

In [ ]:
# dmodel = dict()
# for key in all_res:
#     tmps = key.split("_")
#     model_etype = "_".join(tmps[1:])
#     dmodel.setdefault(model_etype, dict())
#     params = get_params(os.path.join(tmps[0]+"_wandb", key+".yaml"))
#     df = all_res[key].dropna()
#     try:
#         sorted_df = df.sort_values("testauc",ascending=False)
#         top1 = sorted_df[0:1]
#         for param in params:
#             dmodel[model_etype].setdefault(param, set())
#             dmodel[model_etype][param].add(top1[param].tolist()[0])
#     except:
#         continue
# dmodel

In [ ]:
# def get_params(paramsf):
#     lines = open(paramsf).read().split("\n")
#     res = []
#     for i in range(0, len(lines)):
#         cur = lines[i]
#         pre = ""
#         if i > 0:
#             pre = lines[i-1]
#         if pre == "":
#             continue
#         idx = cur.find("values")
#         if idx != -1:
#             key = pre.strip().replace(":", "")
#             res.append(key)
#     return res


In [ ]:
# def download(sweep_dict, dataset_pre):
#     d_data = dict()
#     for key in sweep_dict:
# #         if not key.startswith(dataset_pre):
# #             continue
#         try:
#             df = get_df(key, sweep_dict)
#             d_data[key] = df
#     return d_data

# d_data = download(sweep_dict, "assist2009_")

In [ ]:
# diffs = []
# dataset_name = "assist2009_wandb"
# for key in d_data:
#     d = dict()
#     d.setdefault(key, dict())
#     keys = []
#     paramsf = os.path.join(dataset_name, key+".yaml")
#     params = get_params(paramsf)
#     params = sorted(params)
#     df = d_data[key].dropna()
#     sorted_df = df.sort_values("testauc",ascending=False)
#     top1 = sorted_df[0:1]
#     for i, row in top1[params].iterrows():
#         params_str = "_".join([str(s) for s in row.values])
#     d[key].setdefault(params_str, [])
#     print(params_str)

#     for i, row in sorted_df.iterrows():
#         try:
#             testauc = row["testauc"]
#             testacc = row["testacc"]
#             best_epoch = row["best_epoch"]
#             testauc = round(testauc, 4)
#             testacc = round(testacc, 4)
#             best_epoch = int(best_epoch)
#             curparams = "_".join([str(s) for s in row[params].values])
#     #         print(curparams)
#             if params_str == curparams:
#                 d[key][params_str].append([testauc, row["Name"]])
#         except Exception as e:
#             print(key, row)
#             print(e)
#             import sys
#             sys.exit()
#     min_auc = d[key][params_str][-1][0]
#     max_auc = d[key][params_str][0][0]
#     maxdiff = round(max_auc-min_auc,4)
#     print(min_auc, max_auc, maxdiff, len(d[key][params_str]), df.shape[0])
#     diffs.append(maxdiff)
# sorted(diffs)

In [ ]:
# def download(sweep_dict, dataset_pre):
#     d_data = dict()
#     for key in sweep_dict:
#         if not key.startswith(dataset_pre):
#             continue
#         df = get_df(key, sweep_dict)
#         d_data[key] = df
#     return d_data
# d_data2 = download(sweep_dict, "Statics2011_")

In [ ]:
# diffs = []
# dataset_name = "Statics2011_wandb"
# for key in d_data2:
#     d = dict()
#     d.setdefault(key, dict())
#     keys = []
#     paramsf = os.path.join(dataset_name, key+".yaml")
#     params = get_params(paramsf)
#     params = sorted(params)
#     df = d_data2[key].dropna()
#     sorted_df = df.sort_values("testauc",ascending=False)
#     top1 = sorted_df[0:1]
#     for i, row in top1[params].iterrows():
#         params_keys = "_".join(params)
#         params_str = "_".join([str(s) for s in row.values])
#     d[key].setdefault(params_str, [])
#     print(params_keys)
#     print(params_str)

#     for i, row in sorted_df.iterrows():
#         try:
#             testauc = row["testauc"]
#             testacc = row["testacc"]
#             best_epoch = row["best_epoch"]
#             testauc = round(testauc, 4)
#             testacc = round(testacc, 4)
#             best_epoch = int(best_epoch)
#             curparams = "_".join([str(s) for s in row[params].values])
#     #         print(curparams)
#             if params_str == curparams:
#                 d[key][params_str].append([testauc, row["Name"]])
#         except Exception as e:
#             print(key, row)
#             print(e)
#             import sys
#             sys.exit()
#     min_auc = d[key][params_str][-1][0]
#     max_auc = d[key][params_str][0][0]
#     maxdiff = round(max_auc-min_auc,4)
#     print(min_auc, max_auc, maxdiff, len(d[key][params_str]), df.shape[0])
#     diffs.append(maxdiff)
# sorted(diffs)

In [ ]:
# for i, row in top1[params].iterrows():
#     print("_".join([str(s) for s in row]))
#     print(row)

In [ ]:
a = set([1,2,3])
b = set([1,2,3,4])
a>=a

In [ ]:
tqdm_notebook?